In [1]:
import fasttext
import json
from faq50_adapted import FAQ, extract_word_probs
from topic_word_probs import *
from tfidf_classifier import TFIDF_Classifier
from cs_lemmatizer import *


%load_ext autoreload
%autoreload 2

In [2]:
model_path = "models/cc.cs.300.bin"
model = fasttext.load_model(model_path)

# probs_general = extract_word_probs(model_path=model_path)

### Tests:

In [2]:
path_to_q = "upv_faq/data/FAQ76_questions.xlsx"
path_to_a = "upv_faq/data/FAQ76_answers.xlsx"

probs, n_words = count_word_probs_in_corpuses(path_to_questions=path_to_q, 
                                              path_to_answers=path_to_a)

with open("data_preprocessing/cc.cs.300_probs.json", "r") as wp_file:
    probs_general = json.load(wp_file)

divided_probs = general_to_corpus_probs(probs, probs_general)

print(dict(sorted(divided_probs.items(), key=lambda item: -item[1])))

{'pod': 1.0, 'cena': 0.6074890022706535, ':': 0.5659048059915419, 'zdarma': 0.49365751001500446, 'často': 0.39844489790048426, '2': 0.3128659584656283, '1': 0.2893766009226427, '/': 0.2523183880233372, 'přesně': 0.19289961346157225, 'typ': 0.17953311782630857, 'díl': 0.1261104390942313, 'věc': 0.10989899506400493, '4': 0.09520616956528116, 'ani': 0.09042458505842817, 'délka': 0.09020974383639789, 'řešit': 0.08678243111738208, ';': 0.08640326665060592, 'částečně': 0.08484973897870123, 'software': 0.07243528797032969, 'zrušení': 0.06909737924069727, 'Proč': 0.06847962084399283, '18': 0.06817587098980361, '2007': 0.06368449189220955, 'povinnost': 0.05801895460450519, 'krok': 0.057911094563694136, 'několik': 0.057186634622913304, 'stáhnout': 0.05615864853765753, 'pomoci': 0.05408757596143244, 'místo': 0.05291694727690579, 'veřejně': 0.04855443576415243, 'webu': 0.04561527362094945, 'vlastní': 0.044196919168595916, 'starý': 0.04410980311947217, 'prostor': 0.041361089849810885, 'č': 0.040700

In [16]:
# boost some words: 
print(divided_probs["užitný"])
print(divided_probs["průmyslový"])
divided_probs["užitný"] = 1e-15
divided_probs["průmyslový"] = 1e-15
divided_probs["předpisy"] = 1e-15
divided_probs["najít"] = 1e-15

1e-15
1e-15


In [3]:
q_xslx = "new_questions/M_Q76_questions_V3.xlsx"
a_xslx = "new_questions/M_Q76_answers_V3.xlsx"

faq = FAQ(model, q_xslx, a_xslx, 
          probs=divided_probs, 
          alpha=0.1, 
          rm_stop_words=True, 
          lemm=True)

acc, acc_sec = faq.cross_match_test()
print(f"\nQuestion Cross-Match Accuracy: 1st: {round(acc, 4)}, 2nd: {round(acc_sec, 3)}")

# acc, acc_sec = faq.mean_match_test()
# print(f"\nQuestion Mean-Match Accuracy: 1st: {round(acc, 3)}, 2nd: {round(acc_sec, 3)}")

NameError: name 'divided_probs' is not defined

### Mean match test without inclusion of tested question:

In [ ]:
f, s, t = faq.mean_match_test_disjunctive()
print(f"\033[1m1st right: {f} \033[0m")
print(f"2nd right: {s}\n3rd right: {t}\n-> {round(f+s+t, 3)}")

1st right: 0.48 
2nd right: 0.138
3rd right: 0.072
-> 0.69


In [ ]:
# pouze words from questions:
probs, n_words = count_word_probs_in_corpuses(path_to_questions=path_to_q, path_to_answers=None)
# histogram_of_words(probs, n_words)

In [ ]:
LMTZR.remove_stop_words_from_sentence(["rešerše", "mezi", "jsem", "přihlasit", "jak", "patentem"])

['rešerše', 'přihlasit', 'patentem']

## Weighting with TF-IDF

In [21]:
path_to_q = "upv_faq/data/FAQ76v2_questions.xlsx"
path_to_a = "upv_faq/data/FAQ76v2_answers.xlsx"

# path_to_save = "780_upv_questions/w_probs_in_q_and_a.json"
# probs_old, n_words = count_word_probs_in_corpuses(path_to_save, path_to_questions=path_to_q, path_to_answers=None)

c = TFIDF_Classifier(path_to_q)
test_data = c.structure_data(test_data_percent=1) 
tfidf_matrix, feat_names = c.get_TFIDF_matrix()
probs = get_TFIDF_threshold_probabilities(tfidf_matrix, feat_names)
print(dict(sorted(probs.items(), key=lambda item: -item[1])))

{'stížnost': 0.9509600703709645, 'licence': 0.8727681668398216, 'spor': 0.8628904351830562, 'spisu': 0.8532697090511945, 'výrobek': 0.8174619774319459, 'autorský': 0.8048391890862436, 'tříděný': 0.782033203525745, 'zahraničí': 0.7690841163732133, 'předpis': 0.7674846006256432, 'užitného': 0.7052147792649006, 'námitka': 0.6854220864405053, 'patentovou': 0.6740783458251128, 'přihláškou': 0.6651562735797149, 'věstníku': 0.6634345557813387, 'mezinárodní': 0.6534649864999413, 'užitných': 0.645641451562546, 'rešerši': 0.6335642730462976, 'průmyslový': 0.6275783381825932, 'opis': 0.6149970119721195, 'patentu': 0.6124508423449271, 'ochranný': 0.6111787723784156, 'platnost': 0.6014861696624293, 'označený': 0.5991433789554739, 'zrušený': 0.5987563373734741, 'známka': 0.5977100006698801, 'technický': 0.5856131686995459, 'řešení': 0.5856131686995459, 'přihláška': 0.5852193559063952, 'databáze': 0.5703000004076394, 'symbol': 0.5667670943101568, 'variabilní': 0.5667670943101568, 'právní': 0.56579214

In [9]:
c.most_important_words(top_n=8) # saves to data_preprocessings

In [23]:
""" previous best results without tf-idf weighting:
0.471, _ : alpha=0.23

1st right: 0.481 
2nd right: 0.135
3rd right: 0.065
-> 0.681 : alpha=0.41
"""
faq = FAQ(model, path_to_q, path_to_a, 
          probs=probs,
          alpha=0.11,
          rm_stop_words=True, 
          lemm=True,
          tfidf_weighting=True)

acc, acc_sec = faq.cross_match_test()
print(f"\nQuestion Cross-Match Accuracy: 1st: {round(acc, 3)}, 2nd: {round(acc_sec, 3)}")

# disjunctive, but still the question is included to tfidf matrix...
# f, s, t = faq.mean_match_test_disjunctive()
# print(f"\n\033[1m1st right: {f} \033[0m")
# print(f"2nd right: {s}\n3rd right: {t}\n-> {round(f+s+t, 3)}")


Question Cross-Match Accuracy: 1st: 0.855, 2nd: 0.055


In [40]:
test_question = "Jaká je hlavní výhoda používání patentů na technologie chování zvířat?"
faq.match(test_question, idk_threshold=0.013)

('Jaké jsou zásady pro udělování patentů?', 29)

In [43]:
df = pd.read_excel("data/dont_know_questions.xlsx")
column = df['question']
qs = column.tolist()

got_right = 0
for q in qs:
    question_closest, its_class = faq.match(q, idk_threshold=0.85)
    if its_class == -1:
        got_right += 1
print("Got right:", got_right, "out of", len(qs))

Got right: 43 out of 50


In [ ]:
# ~ 4 minutes
acc, acc_sec = faq.cross_match_test_tfidf_disj()
print(f"\nQuestion Cross-Match Disj Accuracy: 1st: {round(acc, 3)}, 2nd: {round(acc_sec, 3)}")

In [ ]:
# ~ 2 minutes
faq = FAQ(model, q_xslx, a_xslx, probs=probs, alpha=0.3, rm_stop_words=True, lemm=True,
          tfidf_weighting=True)
f, s, t = faq.mean_match_test_disjunctive(leave_one_out_also_tfidf=True)
print(f"\n\033[1m1st right: {f} \033[0m")
print(f"2nd right: {s}\n3rd right: {t}\n-> {round(f+s+t, 3)}")


1st right: 0.5 
2nd right: 0.142
3rd right: 0.071
-> 0.713


## Confused sentences:

In [10]:
LMTZR.tokenize("Jak se máš?")

['jak', 'se', 'máš?']

In [19]:
path_to_q = "upv_faq/data/FAQ76v2_questions.xlsx"
path_to_a = "upv_faq/data/FAQ76v2_answers.xlsx"

faq = FAQ(model, path_to_q, path_to_a,
          rm_stop_words=True, 
          lemm=True,
          )

In [20]:
# Ambiguous matches, disable lemmatizing and stop words removal
path_to_save = "new_questions/same_question_different_answers_on_FAQ76v2.json"
faq.get_same_question_different_answer_pairs(save_path=None)

Same question with different classes:

Q1: Kdo může podat přihlášku ochranné známky? It's idx= 489 , true_class= 26
Q2: Jak mohu podat přihlášku ochranné známky? It's idx=  436 , true_class= 24

Q1: Jak  lze provést rešerši průmyslových vzorů? It's idx= 1273 , true_class= 45
Q2: Jak  lze provést rešerši průmyslového vzoru? It's idx=  1109 , true_class= 42

Q1: Kde lze provést rešerši průmyslových vzorů? It's idx= 1274 , true_class= 45
Q2: Jak  lze provést rešerši průmyslového vzoru? It's idx=  1109 , true_class= 42

Q1: Jak  můžu provést rešerši průmyslových vzorů? It's idx= 1275 , true_class= 45
Q2: Jak  můžu provést rešerši průmyslového vzoru? It's idx=  1111 , true_class= 42

Q1: Kde můžu provést rešerši průmyslových vzorů? It's idx= 1276 , true_class= 45
Q2: Jak  můžu provést rešerši průmyslového vzoru? It's idx=  1111 , true_class= 42

Q1: Jak  mohu provést rešerši průmyslových vzorů? It's idx= 1277 , true_class= 45
Q2: Jak  můžu provést rešerši průmyslového vzoru? It's idx=  1111

{}

In [ ]:
faq.get_most_confused_questions(cos_sim_threshold=0.85) # with mean-match

(788, 79)
(788,)
41     Na jaké stránce jsou uvedeny nejdůležitější pr...
399    Kde mohu najít právní předpisy týkající se pop...
400    Jaké jsou nejdůležitější právní předpisy týkaj...
Name: question, dtype: object


In [ ]:
path_to_save = "780_upv_questions/most_missclassified_class_pairs"
faq.get_most_misclassified_class_pairs(n_of_common_misses=3, save_path=path_to_save)

(19, 45) 4
(26, 24) 3
(26, 25) 3
(30, 55) 4
(31, 69) 4
(33, 34) 4
(35, 65) 3
(36, 57) 5
(40, 4) 3
(42, 44) 3
(43, 51) 3
(43, 47) 3
(45, 55) 3
(49, 51) 5
(52, 51) 4
(53, 62) 4
(58, 68) 6
(65, 55) 3
(71, 51) 3
(77, 3) 5
['19:45', '26:24', '26:25', '30:55', '31:69', '33:34', '35:65', '36:57', '40:4', '42:44', '43:51', '43:47', '45:55', '49:51', '52:51', '53:62', '58:68', '65:55', '71:51', '77:3']


In [28]:
questions_xlsx = "upv_faq/data/FAQ76_questions.xlsx"
# questions_xlsx = "upv_faq/Q50_questions.xlsx"
c = TFIDF_Classifier(questions_xlsx)

In [29]:
test_data = c.structure_data(test_data_percent=0.1) 

c.get_TFIDF_matrix()

(<76x611 sparse matrix of type '<class 'numpy.float64'>'
 	with 1950 stored elements in Compressed Sparse Row format>,
 array(['20', 'advokát', 'aktuální', 'anotace', 'autor', 'autorský',
        'bankovní', 'benelux', 'beneluxu', 'bezplatný', 'biotechnologický',
        'být', 'celý', 'cena', 'ceník', 'cesta', 'charakteristika',
        'chránit', 'chránitelné', 'cílový', 'další', 'databáze', 'datový',
        'definice', 'definovat', 'delá', 'dispozice', 'dlouho', 'doba',
        'dohledat', 'dohoda', 'dojít', 'doklad', 'dokument', 'doplnění',
        'doručit', 'dostat', 'dostupnou', 'dostupný', 'dotaz', 'dozvědět',
        'druh', 'duševní', 'dvůr', 'dát', 'dávat', 'délka', 'dílo',
        'dívat', 'dělat', 'důkaz', 'důsledek', 'důvod', 'elektronicky',
        'elektronické', 'elektronického', 'elektronickém', 'estetický',
        'eu', 'euipo', 'evropský', 'existovat', 'expresní', 'expresních',
        'express', 'forma', 'formulovaný', 'formulář', 'formuář',
        'formální', '

In [32]:
c.classify_test_sentences_list(test_data)

Got right: 0.7227722772277227


In [33]:
test_data = c.structure_data(test_data_percent=1) # without removal from train data
c.get_TFIDF_matrix()
c.classify_test_sentences_list(test_data)

Got right: 0.7645607107601184


In [34]:
c.leave_one_out_test()

2026it [05:35,  6.04it/s]


0.668

## Word embeddings based Edit Distance

In [ ]:
from weed import *

rm_sw = True
lm = True

# q_xslx = "new_questions/FAQ76_questions.xlsx"
# a_xslx = "new_questions/FAQ76_answers.xlsx"
# q_xslx = "780_upv_questions/Q78_questions.xlsx"
# a_xslx = "780_upv_questions/Q78_answers_no_tags.xlsx"
q_xslx = "upv_faq/data/FAQ50_questions.xlsx"
a_xslx = "upv_faq/data/FAQ50_answers.xlsx"

c = TFIDF_Classifier(q_xslx, rm_sw, lm)
test_data = c.structure_data(test_data_percent=1) 
tfidf_matrix, feat_names = c.get_TFIDF_matrix()
probs = get_TFIDF_threshold_probabilities(tfidf_matrix, feat_names)

# Or from frequencies - change to tfidf_weighting=False
path_to_save = "w_probs_in_questions.json"
probs, n_words = count_word_probs_in_corpuses(path_to_save, path_to_questions=q_xslx, path_to_answers=None)

weed = WEED(model, q_xslx, a_xslx, probs=probs, alpha=1, 
            lemm=lm, rm_stop_words=rm_sw, sigma=0.9,
            tfidf_weighting=False)
print(weed.nearest_question_test_weed())

0.9448398576512456


: 

In [ ]:
"Finding best sigma"
best_alpha_so_far = None
best_acc = 0
for sigma in np.arange(0.0, 1.0, 0.1):
    weed = WEED(model, q_xslx, a_xslx, probs=probs, alpha=0.05, 
                lemm=lm, rm_stop_words=rm_sw, sigma=sigma)
    acc = weed.nearest_question_test_weed()
    print("\n", acc)
    if acc > best_acc:
        best_acc = acc
        best_alpha_so_far = sigma

        print("S:", best_alpha_so_far)

Average number of words in a tokenized sentence: 8.803

 0.5101522842639594
S: 0.0
Average number of words in a tokenized sentence: 8.803

 0.5101522842639594
Average number of words in a tokenized sentence: 8.803

 0.5177664974619289
S: 0.2
Average number of words in a tokenized sentence: 8.803

 0.5114213197969543
Average number of words in a tokenized sentence: 8.803

 0.5215736040609137
S: 0.4
Average number of words in a tokenized sentence: 8.803

 0.5279187817258884
S: 0.5
Average number of words in a tokenized sentence: 8.803

 0.5342639593908629
S: 0.6000000000000001
Average number of words in a tokenized sentence: 8.803

 0.5317258883248731
Average number of words in a tokenized sentence: 8.803

 0.5215736040609137
Average number of words in a tokenized sentence: 8.803

 0.5177664974619289


In [ ]:
"Finding best alpha"
best_alpha_so_far = None
best_acc = 0
for alpha in np.arange(0.01, 0.6, 0.022):
    weed = WEED(model, q_xslx, a_xslx, probs=probs, alpha=alpha, 
                lemm=lm, rm_stop_words=rm_sw)
    acc = weed.nearest_question_test_weed()
    if acc > best_acc:
        best_acc = acc
        best_alpha_so_far = alpha

        print(best_alpha_so_far)
        print(best_acc)

0.01
0.7764067127344522
0.032
0.7828232971372162


## BERT-like

In [1]:
import fasttext
import json
from faq50_adapted import FAQ, extract_word_probs
from topic_word_probs import *
from tfidf_classifier import TFIDF_Classifier
from cs_lemmatizer import *
from bert_like_models.slavic_bert_tests import SlavicBERT
from weed import WEED
from bert_like_models.robeczech import Robeczech

%load_ext autoreload
%autoreload 2

### Slavic BERT


In [5]:
q_xslx = "upv_faq/data/FAQ50_questions.xlsx"

dir_to_models_bert = "bert_like_models/models"
model = SlavicBERT(dir_to_models=dir_to_models_bert,
                   )

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert_like_models/models/bg_cs_pl_ru_cased_L-12_H-768_A-12_pt_v1/pytorch_model.bin and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
c = TFIDF_Classifier(q_xslx, rm_sw=False, lm=False)
test_data = c.structure_data(test_data_percent=1,
                             tokenizer=model.tokenizer.tokenize) 
tfidf_matrix, feat_names = c.get_TFIDF_matrix()
probs = get_TFIDF_threshold_probabilities(tfidf_matrix, feat_names)

# probs, n_words = count_word_probs_in_corpuses(path_to_questions=q_xslx,
                                    #  tokenizer=model.tokenizer.tokenize)

faq = FAQ(model, q_xslx, 
          slBert=True, 
          rm_stop_words=False,
          probs=probs,
          alpha=0.5,
          lemm=False,
          tfidf_weighting=True,
        #   sigma=0.7,
          )

In [ ]:
print(faq.mean_db.shape)

(50, 768)


In [ ]:
# faq.sigma = 0.9
# print(faq.nearest_question_test_weed())

0.7833168805528135


In [ ]:
# for FAQ
acc, acc_sec = faq.cross_match_test()
print(f"Cross-Match Accuracy: 1st: {round(acc, 3)}")

# disjunctive, but still the question is included to tfidf matrix...
f, s = faq.mean_match_test()
print("Mean-Match Accuracy:", f)

# f, s, t = faq.mean_match_test_disjunctive() # raw: 0.69
# print("Mean-Match Disj Accuracy:", f)

Cross-Match Accuracy: 1st: 0.867
Mean-Match Accuracy: 0.792


### ROBECZECH

In [3]:
m = Robeczech()

In [63]:
text = "Co chrání autorské právo?"
text2 = "Co se stane po podání stížnosti?"
# text2 = "Kam se obrátit s dotazy k autorským právům?"

# text = "Auto jede po cestě do Ostravy"
# text2 = "Pomeranče jsou nejlepší ovoce"

embs = m.get_mean_sentence_embedding(text, mean=True)
print(embs.shape)
embs2 = m.get_mean_sentence_embedding(text2, mean=True)
print(embs2.shape)

print("Cosine:", embs @ embs2)

(768,)
(768,)
Cosine: 0.9208753


In [6]:
q_xslx = "upv_faq/data/FAQ79_questions.xlsx"

model = Robeczech()
# print(model.model)

In [56]:
# c = TFIDF_Classifier(q_xslx, rm_sw=False, lm=False)
# test_data = c.structure_data(test_data_percent=1,
#                              tokenizer=model.tokenizer.tokenize) 
# tfidf_matrix, feat_names = c.get_TFIDF_matrix()
# probs = get_TFIDF_threshold_probabilities(tfidf_matrix, feat_names)

# probs, n_words = count_word_probs_in_corpuses(path_to_questions=q_xslx,
#                                      tokenizer=model.tokenizer.tokenize)

faq = WEED(model, q_xslx, 
          slBert=True, 
          rm_stop_words=False,
          lemm=False,
        #   probs=probs,
        #   tfidf_weighting=True,
          sigma=1,
          )

In [54]:
acc = faq.nearest_question_test_weed()
print("WEED acc:", acc)

WEED acc: 0.4073604060913706


In [57]:
# for FAQ
acc, acc_sec = faq.cross_match_test()
print(f"Cross-Match Accuracy: 1st: {round(acc, 3)}")

# disjunctive, but still the question is included to tfidf matrix...
f, s = faq.mean_match_test()
print("Mean-Match Accuracy:", f)

# f, s, t = faq.mean_match_test_disjunctive() # raw: 0.69
# print("Mean-Match Disj Accuracy:", f)

Cross-Match Accuracy: 1st: 0.412
Mean-Match Accuracy: 0.471


In [68]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('ufal/robeczech-base')

No sentence-transformers model found with name /home/micha/.cache/torch/sentence_transformers/ufal_robeczech-base. Creating a new one with MEAN pooling.


In [69]:
text = "Co chrání autorské právo?"
text2 = "Co se stane po podání stížnosti?"
# text2 = "Kam se obrátit s dotazy k autorským právům?"

# text = "Auto jede po cestě do Ostravy"
# text2 = "Pomeranče jsou nejlepší ovoce"

emb = model.encode(text)
print(emb.shape)
emb2 = model.encode(text2)
print(emb2.shape)

print("Cosine:", (emb@emb2) / (np.linalg.norm(emb)*np.linalg.norm(emb2)))

(768,)
(768,)
Cosine: 0.9208754


## Tests with answer @ question with BERT

In [13]:
# db of mean answer embeddings
path_to_a = "upv_faq/data/FAQ76_answers.xlsx"
# path_to_a = "data_preprocessing/keywords_from_q.xlsx"
path_to_q = "upv_faq/data/FAQ76_questions.xlsx"

df = pd.read_excel(path_to_a)
column_data = df['answer'].tolist()
classes = df['class'].tolist()
max_len = 0
max_a = ""
all_ans_embs = []
already_have = set()
for i, row in enumerate(column_data):
    if classes[i] in already_have:
        continue
    already_have |= {classes[i]}
    all_ans_embs.append(m.get_mean_sentence_embedding(row, mean=True))
all_ans_embs = np.array(all_ans_embs)
print(all_ans_embs.shape)

(76, 768)


In [14]:
faq = WEED(m, path_to_q, 
            answers_path=path_to_a,
            slBert=True,
            rm_stop_words=False,
            lemm=False,
            sigma=0.6,
            )

In [15]:
f = faq.nearest_question_test_weed(with_answers=True)
print("Accuracy:", f)

Accuracy: 0.3188548864758144


## KeyBERT weights


In [1]:
import fasttext
import json
from faq50_adapted import FAQ, extract_word_probs
from topic_word_probs import *
from tfidf_classifier import TFIDF_Classifier
from cs_lemmatizer import *
from bert_like_models.slavic_bert_tests import SlavicBERT
from weed import WEED
from bert_like_models.robeczech import Robeczech

%load_ext autoreload
%autoreload 2

In [2]:
model_path = "models/cc.cs.300.bin"
model = fasttext.load_model(model_path)

In [3]:
q_xlsx = "upv_faq/data/FAQ76_questions.xlsx"
path_to_save = "bert_like_models/weights_with_KeyBERT.json"
sw = True
lm = True

df = pd.read_excel(q_xlsx)
test_data = df['question'].to_list()

# c = TFIDF_Classifier(q_xlsx, rm_sw=sw, lm=lm)
# test_data = c.structure_data(test_data_percent=1)

# probs = get_weights_with_KeyBERT(test_data, sw=sw, lm=lm, 
#                                  path_to_save=path_to_save)

with open(path_to_save, "r") as wp_file:
    probs = json.load(wp_file)

print("Number of words:", len(probs))
print(dict(sorted(probs.items(), key=lambda item: -item[1])))

Number of words: 575
{'průmyslový': 1.0, 'právo': 1.0, 'zahraničí': 1.0, 'přihlašování': 1.0, 'přihláška': 1.0, 'přihlásit': 1.0, 'mezinárodní': 1.0, 'ochranný': 1.0, 'známka': 1.0, 'patentu': 1.0, 'patent': 1.0, 'vynález': 1.0, 'dlouho': 1.0, 'zaplatit': 1.0, 'podávat': 1.0, 'právní': 1.0, 'předpis': 1.0, 'přehled': 1.0, 'úřad': 1.0, 'patentoví': 1.0, 'zástupkyně': 1.0, 'spisu': 1.0, 'nahlížení': 1.0, 'patentové': 1.0, 'odvolat': 1.0, 'rozklad': 1.0, 'porušený': 1.0, 'vymáhání': 1.0, 'náhrada': 1.0, 'škoda': 1.0, 'spor': 1.0, 'služba': 1.0, 'výrobek': 1.0, 'databáze': 1.0, 'rešerše': 1.0, 'ceník': 1.0, 'rešeršní': 1.0, 'zahraniční': 1.0, 'podmínka': 1.0, 'produkt': 1.0, 'novost': 1.0, 'patentovatelnosti': 1.0, 'odkaz': 1.0, 'patentová': 1.0, 'variabilní': 1.0, 'symbol': 1.0, 'poplatků': 1.0, 'souhrnný': 1.0, 'souhrn': 1.0, 'obecně': 1.0, 'elektronické': 1.0, 'systém': 1.0, 'složený': 1.0, 'utajení': 1.0, 'obchodní': 1.0, 'tajemství': 1.0, 'název': 1.0, 'lokální': 1.0, 'sleva': 1.0, '2

In [13]:
faq = WEED(model, q_xlsx, 
          probs=probs,
          alpha=0.3,
          rm_stop_words=sw, 
          lemm=lm,
          tfidf_weighting=True,
          sigma=1,
          )

faq.nearest_question_test_weed()

0.7847976307996052

: 

In [10]:
faq = FAQ(model, q_xlsx, 
          probs=probs,
          alpha=0.02,
          rm_stop_words=sw, 
          lemm=lm,
          tfidf_weighting=True,
          )

acc, acc_sec = faq.cross_match_test()
print(f"\nQuestion Cross-Match Accuracy: 1st: {round(acc, 3)}, 2nd: {round(acc_sec, 3)}")



Question Cross-Match Accuracy: 1st: 0.835, 2nd: 0.061
